(xarray_for_arviz)=
# Introduction to xarray, InferenceData, and netCDF for ArviZ

While ArviZ supports plotting from familiar data types, such as dictionaries and NumPy arrays, there are a couple of data structures central to ArviZ that are useful to know when using the library. 

They are  

* {class}`xarray:xarray.Dataset`
* {class}`arviz.InferenceData`
* {ref}`netCDF <netcdf>` 


## Why more than one data structure?
[Bayesian inference](https://en.wikipedia.org/wiki/Bayesian_inference) generates numerous datasets that represent different aspects of the model. For example, in a single analysis, a Bayesian practitioner could end up with any of the following data.



* Prior Distribution for N number of variables
* Posterior Distribution for N number of variables
* Prior Predictive Distribution
* Posterior Predictive Distribution
* Trace data for each of the above
* Sample statistics for each inference run
* Any other array like data source

For more detail, see the `InferenceData` structure specification {ref}`here <schema>`.


## Why not Pandas Dataframes or NumPy Arrays?
Data from [probabilistic programming](https://en.wikipedia.org/wiki/Probabilistic_programming) is naturally high dimensional. To add to the complexity ArviZ must handle the data generated from multiple Bayesian modeling libraries, such as PyMC3 and PyStan. This application is handled by the *xarray* package quite well. The xarray package lets users manage high dimensional data with human readable dimensions and coordinates quite easily.

![InferenceData Structure](InferenceDataStructure.png)  

Above is a visual representation of the data structures and their relationships. Although it seems more complex at a glance, the ArviZ devs believe that the usage of *xarray*, `InferenceData`, and *netCDF* will simplify the handling, referencing, and serialization of data generated during Bayesian analysis.  



## An introduction to each
To help get familiar with each, ArviZ includes some toy datasets. You can check the different ways to start an `InferenceData` {ref}`here <creating_InferenceData>`. For illustration purposes, here we have shown only one example provided by the library. To start an `az.InferenceData`, sample can be loaded from disk.

In [6]:
# Load the centered eight schools model
import arviz as az

data = az.load_arviz_data("centered_eight")
data

Inference data with groups:
	> posterior
	> posterior_predictive
	> sample_stats
	> prior
	> observed_data

In this case the `az.InferenceData` object contains both a posterior predictive distribution and the observed data, among other datasets. Each group in `InferenceData` is both an attribute on `InferenceData` and itself a `xarray.Dataset` object. 

In [7]:
# Get the posterior dataset
posterior = data.posterior
posterior

<xarray.Dataset>
Dimensions:  (chain: 4, draw: 500, school: 8)
Coordinates:
  * chain    (chain) int64 0 1 2 3
  * draw     (draw) int64 0 1 2 3 4 5 6 7 8 ... 492 493 494 495 496 497 498 499
  * school   (school) object 'Choate' 'Deerfield' ... "St. Paul's" 'Mt. Hermon'
Data variables:
    mu       (chain, draw) float64 -3.477 -2.456 -2.826 ... 4.597 5.899 0.1614
    theta    (chain, draw, school) float64 1.669 -8.537 -2.623 ... 10.59 4.523
    tau      (chain, draw) float64 3.73 2.075 3.703 4.146 ... 8.346 7.711 5.407
Attributes:
    created_at:                 2019-06-21T17:36:34.398087
    inference_library:          pymc3
    inference_library_version:  3.7

In our eight schools model example, the posterior trace consists of 3 variables and approximately over 4 chains. In addition, it is a hierarchical model where values for the variable `theta` are associated with a particular school. 

According to the xarray's terminology: 
* Data variables are the actual values generated from the MCMC draws
* Dimensions are the axes on which refer to the data variables
* Coordinates are pointers to specific slices or points in the `xarray.Dataset`

Observed data from the eight schools model can be accessed through the same method.

In [8]:
# Get the observed xarray
observed_data = data.observed_data
observed_data

<xarray.Dataset>
Dimensions:  (school: 8)
Coordinates:
  * school   (school) object 'Choate' 'Deerfield' ... "St. Paul's" 'Mt. Hermon'
Data variables:
    obs      (school) float64 28.0 8.0 -3.0 7.0 -1.0 1.0 18.0 12.0
Attributes:
    created_at:                 2019-06-21T17:36:34.491909
    inference_library:          pymc3
    inference_library_version:  3.7

It should be noted that the observed dataset contains only 8 data variables. Moreover, it doesn't have a chain and draw dimension or coordinates unlike posterior. This difference in sizes is the motivating reason behind `InferenceData`. Rather than force multiple different sized arrays into one array, or have users manage multiple objects corresponding to different datasets, it is easier to hold references to each `xarray.Dataset` in an `InferenceData` object.

(netcdf)=

## NetCDF
[NetCDF](https://www.unidata.ucar.edu/software/netcdf/) is a standard for referencing array oriented files. In other words, while `xarray.Dataset`s, and by extension `InferenceData`, are convenient for accessing arrays in Python memory, *netCDF* provides a convenient mechanism for persistence of model data on disk. In fact, the netCDF dataset was the inspiration for `InferenceData` as netCDF4 supports the concept of groups. `InferenceData` merely wraps `xarray.Dataset` with the same functionality.

Most users will not have to concern themselves with the *netCDF* standard but for completeness it is good to make its usage transparent. It is also worth noting that the netCDF4 file standard is interoperable with HDF5 which may be familiar from other contexts.

Earlier in this tutorial `InferenceData` was loaded from a *netCDF* file

In [9]:
data = az.load_arviz_data("centered_eight")

Similarly, the `InferenceData` objects can be persisted to disk in the netCDF format

In [10]:
data.to_netcdf("eight_schools_model.nc")

'eight_schools_model.nc'

# Additional Reading
Additional documentation and tutorials exist for xarray and netCDF4. Check the following links:

## xarray
* For getting to know xarray, check [xarray documentation](http://xarray.pydata.org/en/stable/why-xarray.html)
* Feel free to watch the Q/A session about xarray at [xarray lightning talk at SciPy 2015](https://www.youtube.com/watch?v=X0pAhJgySxk&t=949s)

## NetCDF
* Get to know the introduction of netCDF at the official website  of [NetCDF  documentation](https://www.unidata.ucar.edu/software/netcdf/docs/.)
* Netcdf4-python library is a used to read/write netCDF files in both netCDF4 and netCDF3 format. Learn more about it by visitng its API documentation at [NetCDF4 API documentation](http://unidata.github.io/netcdf4-python/)
* xarray provides direct serialization and IO to netCDF format. Learn how to read/write netCDF files directly as xarray objects at {ref}`NetCDF usage in xarray <xarray:io.netcdf>`
* Check how to read/write netCDF4 files with HDF5 and vice versa at [NetCDF interoperability with HDF5](https://www.unidata.ucar.edu/software/netcdf/docs/interoperability_hdf5.html)
